# Derin Öğrenme ile Kedi ve Köpek Sınıflandırma

## İçindekiler
1. Giriş
2. Temel Bağımlılıkları Belirleme
3. Veri Setinde Etiketlerin Okunması
4. Veri Görselleştirme
5. Veri bölünmesi ve Veri Dağıtımı
6. Vanilla CNN ile Sınıflandırma
7. Veri Setini Büyütmek ve Öğrenme Oranı Çizelgesi (Learning Rate Schedule)
8. ResNet50 ile Sınıflandırma
9. Test Verisi ile Son Tahminler


# 1. Giriş:

Problemimiz, kedi ve köpek fotoğraflarını etiketlenmiş görüntü verilerine göre sınıflandırmaktır. Veri setimizde train kısmı için 25.000, test kısmı için 12500 kedi ve köpek fotoğrafı vardır. Köpekler 1, kediler 0 olarak etiketlenmiştir. En yüksek doğruluğa ulaşmak için farklı derin öğrenme teknikleri kullanılmıştır.

# # 1.1 Convolutional Neural Network (CNN)
CNN resim tanıma için kullanılan çok etkili bir mekanizmadır.


Cnn verilen görüntüleri ayırt etmek için bir uçağı uçak ya da yılanı yılan haline getiren benzersiz özellikleri kullanır. Aslında bu süreç beynimizde de bilinçsizce oluyor.

Örneğin, bir uçak resmine baktığımızda, iki kanat, motor, pencere gibi özellikleri birbirinden ayırarak uçağı tanımlayabiliriz. Cnn de aynı şeyi yapar, ancak daha önce eğriler ve kenarlar gibi alt düzey özellikleri tespit ederler ve daha soyut kavramlara kadar bunları oluştururlar.

# # 1.2 Evrişimsel Sinir Ağlarının Yapısı
Konuştuğumuz işlevselliği elde etmek için, Cnn görüntüyü çeşitli katmanlarla işler. Bu katmanlara ve amaçlarına genel bir bakış yapalım:

**Convolutional Layer** — Özellikleri saptamak için kullanılır
Convolutional (evrişim katmanı) CNN algoritmalarında görüntüyü ele alan ilk katmandır. Bilindiği üzere görseller aslında içlerinde belirli değerler taşıyan piksellerden oluşan matrislerdir. Evrişim katmanında da orijinal görsel boyutlarından daha küçük bir filtre görselin üzerinde gezer ve bu görsellerden belirli özellikleri yakalamaya çalışır.


**Pooling (Downsampling) Layer** — Ağırlık sayısını azaltır ve uygunluğu kontrol eder.
Evrişimli katman gibi pooling (havuzlama) katmanı da boyutsallığı azaltma amacındadır. Bu sayede hem gereken işlem gücü azalır hem de yakalanan gereksiz özellikler yok sayılarak daha önemli özelliklere odaklanılır.

**Fully-Connected Layer** — Sınıflamada kullanılan Standart Sinir Ağı
Fully Connected katmanda birkaç kez evrişimli katmandan ve pooling katmanından geçen ve matris halinde olan görselimiz düz bir vektör haline getirilir.


# 2.Temel Bağımlılıkları Belirleme

In [ ]:

import zipfile
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from sklearn.model_selection import train_test_split

# 3. Veri Setinde Etiketlerin Okunması


In [ ]:
print('Veriseti zipten çıkarmak')
zip_files = glob.glob('/kaggle/input/dogs-vs-cats/*.zip')


print('{} tane dosya bulundu'.format(len(zip_files)))
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as Z:
        Z.extractall('data')
    print ('{} çıkarma işlemi bitti'.format(file.split('/')[-1]))
          
  

In [ ]:
# Resim sayıları

train_dir = '/kaggle/working/data/train/'
test_dir  = '/kaggle/working/data/test1/'

print('Train dosyaları {} adet görsel'.format(len(os.listdir(train_dir))))
print('Test dosyaları  {} adet görsel'.format(len(os.listdir(test_dir))))            

In [ ]:


def category(path): 
    label = []
    for file in os.listdir(path):
        lab = file.split('.')[0]
        label.append(lab)
    return label



def filename(path):
    fname = []
    for file in os.listdir(path):
        fname.append(file)
    return fname


In [ ]:
x_train = filename(train_dir) 
y_train = category(train_dir)
x_test = filename(test_dir)


print('Dataframeler oluşturuluyor...')
total_df = pd.DataFrame({ 'filename': x_train, 'category': y_train})
sub_df = pd.DataFrame({'filename': x_test})

print('Dataframeler oluşturuldu')

print('Train kısmının içerisindeki tüm resimler total_df dataframe i hazırlanırken kullanılır')
print('Test kısmının içerisindeki kısımlar sub_df (submission dataframe) hazırlanırken kullanılır')



# 4. Veri Görselleştirme

In [ ]:
print('Bu fonksiyon kategorilerin ID lerini, resimlerin dosya yolunu ve kategorileri verir.')
def img_path(directory):
    paths = []
    cate = []
    ID_no = []
    for file in os.listdir(directory):
        path = os.path.join(directory, file)
        paths.append(path)
        cate.append(file.split('.')[0])
        ID_no.append(file.split('.')[1])
    return ID_no, paths, cate


In [ ]:
ID_no, img_paths, train_images = img_path(train_dir)

print('Dataframe içerisindeki verilerin görselleştirilmesi')
visual_df = pd.DataFrame({'ID_no':ID_no,'Category':train_images, 'img_paths': img_paths})

#dataframe'e bir bakış
visual_df.head(10)

In [ ]:

print('showImages fonksiyonu resimlerin galeri şeklinde gözükmesini sağlayan fonksiyondur')

print('num_row galeri görünümünün satır sayısını verir')
print('num_col galeri görünümünün kolon sayısını verir')
def showImages(num_row,num_col,data, what ):
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    cat_df = data[data['Category'] == 'cat']
    dog_df = data[data['Category'] == 'dog']

    if what == 'dog':
        X = dog_df['img_paths']
        Y = dog_df['ID_no']
    elif what == 'cat':
        X = cat_df['img_paths']
        Y = cat_df['ID_no']
    else:
        X = data['img_paths']
        Y = data['ID_no']


    from sklearn.utils import shuffle
    (X_rand, Y_rand) = shuffle(X, Y)
    
    fig, axes = plt.subplots(num_row,num_col,figsize = (12,12))
    fig.suptitle(' ',fontsize=10)

    axes = axes.ravel()
    for i in range(0, num_row*num_col):
        x = load_img(X_rand.iloc[i],target_size= (150, 150))
        axes[i].imshow(x)
        axes[i].set_title("{}".format(Y_rand.iloc[i]))
        axes[i].axis('off')
        plt.subplots_adjust(wspace =0)
    fig.tight_layout()
    print("ID numaralarıyla beraber {} adet {} örneği".format((num_row * num_col),what))
    
    return

In [ ]:
showImages( 10,10,visual_df, 'dog')

In [ ]:
showImages(10, 10, visual_df, 'cat')

In [ ]:
showImages(10,10,visual_df, 'Dogs and Cat')

# 5. Veri Bölünmesi ve Veri Dağıtımı

In [ ]:
train_valid_df, test_df = train_test_split(total_df, test_size = 0.04)
train_df, valid_df = train_test_split(train_valid_df, test_size = 0.2)


train_images = train_df.shape[0]
valid_images = valid_df.shape[0]
test_images = sub_df.shape[0]

print('Veri setinden training için ayrılan görsel miktarı {}'.format(train_images))
print('Veri setinden validation için ayrılan görsel miktarı {}'.format(valid_images))
print('Veri setinden test için ayrılan görsel miktarı {}'.format(test_images))


In [ ]:
# Veri dağılım grafikleri

data  = [train_df['category'] ,  valid_df['category'], test_df['category']]

fig, axis = plt.subplots(1,2, figsize = (25,6))
axis = axis.ravel()
sns.countplot(train_df['category'], ax = axis[0])
axis[0].set_title('Train verisinin dağılımı')
axis[0].set_xlabel('Classes')
sns.countplot(valid_df['category'], ax = axis[1])
axis[1].set_title('Validation verisinin dağılımı')
axis[1].set_xlabel('Classes')
plt.show

# 6. Vanilla CNN ile Sınıflandırma


Veri setimiz eşit olarak dağıtılmıştır. Bu çalışmada ilk olarak önceden eğitilmiş bir model kullanılacaktır. Bu model daha da geliştirilebilir. Çalışmamız [keras applcations](https://keras.io/api/applications/) içerisindeki ImageNet çalışması ile benzerlik gösterdiğinden dolayı 2 etikete göre sınıflandırmak için içerisindeki herhangi bir model kullanılabilir. Sıfırdan özel bir CNN modeli ile daha yüksek bir doğruluk elde etmeye çalışmak o kadar pratik değildir. 

Öncelikle ne kadar iyi performans gösterebileceğini görmek için veri artırma işlemleri olmadan basit bir vanilla network'ü ile başlayalım. Daha sonra diğer tekniğe geçeceğiz.

In [ ]:
img_size = 224
batch_size = 128


In [ ]:
# dataframeiterators without data agumnetation

train_map = ImageDataGenerator()
valid_map = ImageDataGenerator()
test_map =  ImageDataGenerator()

        
#Creating a dataframe iterators for fitting
vani_train_data = train_map.flow_from_dataframe(
            train_df,train_dir,
            x_col = 'filename',
            y_col = 'category',
            target_size = (img_size, img_size),
            batch_size = batch_size,
            class_mode = 'categorical')

vani_valid_data = valid_map.flow_from_dataframe(
             valid_df, train_dir,
             x_col = 'filename',
             y_col = 'category',
             target_size = (img_size, img_size),
             batch_size = batch_size,
             class_mode = 'categorical')


vani_test_data = test_map.flow_from_dataframe(
             test_df, train_dir,
             x_col = 'filename',
             y_col = None,
             target_size = (img_size, img_size),
             batch_size = batch_size,
             class_mode = None,
             shuffle = False)





**Sıralı katmanlardan oluşan Sequential modeli oluşturulur.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
    
    
vani_model = Sequential()
vani_model.add(Conv2D(16, (3,3), activation = 'relu', padding = 'same', input_shape = (224,224,3)))
vani_model.add(Conv2D(16, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
vani_model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape = (224,224,3)))
vani_model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
vani_model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2),strides=(2,2)))
vani_model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2),strides=(2,2)))
vani_model.add(Dropout(0.3))
vani_model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2),strides=(2,2)))
vani_model.add(Dropout(0.3))
vani_model.add(Flatten())
vani_model.add(Dense(512, activation = 'relu'))
vani_model.add(Dropout(0.5))
vani_model.add(Dense(2, activation = 'softmax'))

vani_model.summary()




In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

plot_model(vani_model, to_file='vani_model.png')
SVG(model_to_dot(vani_model).create(prog='dot', format='svg'))

In [ ]:
loss = 'categorical_crossentropy'
opt = tf.keras.optimizers.Adam(learning_rate= 0.0001,beta_1=0.9, beta_2=0.999,epsilon=1e-07)
metrics = ['accuracy']

vani_model.compile(loss = loss, optimizer = opt, metrics = metrics)

vani_history = vani_model.fit(vani_train_data, epochs = 15,
                          validation_data = vani_valid_data,
                          validation_steps= valid_images//batch_size,
                          steps_per_epoch= train_images//batch_size)


In [ ]:
hist1 = vani_history.history
fig = plt.figure(figsize = (15,5))

Epochs =  range(len(hist1['loss']))

fig.add_subplot(1, 2 ,1)
sns.lineplot(x = Epochs, y = hist1['val_loss'], label = 'Validation Loss')
sns.lineplot(x = Epochs, y = hist1['loss'], label = 'Training_loss')

fig.add_subplot(1,2,2)
sns.lineplot(x = Epochs, y = hist1['val_accuracy'], label = 'Validation accuracy')
sns.lineplot(x = Epochs, y = hist1['accuracy'], label = 'Training_accuracy')


In [ ]:

import warnings
warnings.filterwarnings('ignore')

vani_pred = vani_model.predict_generator(vani_test_data)
test_df['vani_pred'] = np.argmax(vani_pred, axis = -1)
labels = dict((v,k) for k,v in vani_train_data.class_indices.items())

test_df['vani_pred'] = test_df['vani_pred'].map(labels)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):


    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    if sum_stats:
        accuracy  = np.trace(cf) / float(np.sum(cf))

        if len(cf)==2:
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    if figsize==None:
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        categories=False


    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

vani_cf_matrix = confusion_matrix(test_df['category'],test_df['vani_pred'])


labels = [ 'True Neg','False Pos','False Neg','True Pos']
categories = ['Cat', 'Dog']
make_confusion_matrix(vani_cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      title = 'Vanila CNN comfusion matrix')

vani_matrix = classification_report(test_df['category'],test_df['vani_pred'])
print('Classification report : \n',vani_matrix)

# 7. Veri Setini Büyütmek ve Öğrenme Oranı Çizelgesi (Learning Rate Schedule)

In [ ]:
def data_argumentation_show(n, grid_size):
    sample_aug_map = ImageDataGenerator(
            zoom_range = 0.1,
            rotation_range = 25,
            horizontal_flip = True,
            height_shift_range =0.2,
            width_shift_range = 0.2,
            fill_mode='nearest',
            rescale = 1/255)
    sample_data = sample_aug_map.flow_from_dataframe(
            (train_df.sample(n)),
            train_dir,
            x_col = 'filename',
            y_col = 'category',
            target_size = (img_size, img_size),
            class_mode = 'categorical')
  
  #subplot grid 
    plt.figure(figsize = (15,15))
    for i in range(0,grid_size*grid_size):
        plt.subplot(grid_size,grid_size, i+1)
        for x,y in sample_data:
            img = x[0]
            plt.imshow(img)
            break
            plt.tight_layout()
            plt.show()

    return 



In [ ]:

data_argumentation_show(1, 5)

In [ ]:
data_argumentation_show(2, 5)

In [ ]:
print('öğrenme hızı çizelgesi (learning rate schedule) için bir zayıflama öğrenme oranı (decay learning rate) belirleme')
epoch = 50
learning_rate = 3e-5 
lr_start = 0.00000001
lr_min = 0.000001
lr_max = 3e-5 
lr_rampup_epochs = 1
lr_sustain_epochs = 1
lr_exp_decay = .8

def lrfn(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
    return lr
    



In [ ]:
epochs = 20
epochs_range = [i for i in range(50 if epochs<50 else epochs)]
learn_rate = [lrfn(x) for x in epochs_range]


print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(learn_rate[0], max(learn_rate), learn_rate[-1]))


fig = plt.figure()
plt.plot(epochs_range, learn_rate, 'sr-') 
plt.xlabel('Range of epochs')
plt.ylabel('Learning rate in 10^-5')
plt.title('Learning Rate Schedule')


print('Bu çizelge(schedule), öğrenme hızının maksimum düzenleme periyodunun olmasını ve üssel toplamının azaltmasını sağlar.')

# 8. ResNet50 ile Sınıflandırma

ResNet, 2015 yılında He Kaiming, Sun Jian ve Microsoft Research Asia'dan diğerleri tarafından önerilen bir ağ yapısıdır ve ILSVRC-2015 sınıflandırma görevinde birinci olmuştur . Aynı zamanda ImageNet algılama, ImageNet yerelleştirme, COCO algılama ve COCO segmentasyon görevlerinde birinci olmuştur. 

Ağ modelinin gerçek anlamda derinleşmeye başladığı kendinden önceki modellerden farklı bir mantığı barındıran ResNet; artık değerlerin (residual value) sonraki katmanlara besleyen blokların (residual block) modele eklenmesiyle oluşmaktadır. ResNet bu özelliği ile klasik bir model olmaktan çıkmaktadır ve modelin performansını daha iyi bir hale getirmektedir.


In [ ]:


from keras.applications.resnet50 import preprocess_input

train_aug_map = ImageDataGenerator(
                    rotation_range=10,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    preprocessing_function = preprocess_input)
res_train_data = train_aug_map.flow_from_dataframe(
            train_df, train_dir,
            x_col = 'filename',
            y_col = 'category',
            target_size = (img_size, img_size),
            batch_size = batch_size,
            class_mode = 'categorical')


valid_aug_map = ImageDataGenerator(preprocessing_function = preprocess_input)

res_valid_data = valid_aug_map.flow_from_dataframe(
             valid_df, train_dir,
             x_col = 'filename',
             y_col = 'category',
             target_size = (img_size, img_size),
             batch_size = batch_size,
             class_mode = 'categorical')




test_aug_map = ImageDataGenerator(preprocessing_function = preprocess_input)

res_test_data = test_aug_map.flow_from_dataframe(
             test_df, train_dir,
             x_col = 'filename',
             y_col = None,
             class_mode = None,
             target_size = (img_size, img_size),
             shuffle = False)
           


In [ ]:
from keras.applications import resnet
from keras.applications.resnet50 import preprocess_input
from keras.layers import  *
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers

from keras import backend as K
K.clear_session()

#loading resent 
resNet = resnet.ResNet50(weights = 'imagenet',
                        include_top = False,
                        input_shape = (224,224, 3))

resNet.trainable = False # Freeze layers
resNet_model = Sequential([
        resNet,
        Flatten(),
        Dense(1024, activation = 'relu'),
        Dropout(0.4),
        Dense(2, activation = 'softmax')])
     

optimizer = optimizers.Adam(1e-5)

resNet_model.summary()

In [ ]:
plot_model(resNet_model, to_file='resNet_model.png')
SVG(model_to_dot(resNet_model).create(prog='dot', format='svg'))

In [ ]:
print('Erken durdurma faktörünün ve öğrenme oranı çizelgesinin (learning rate schedule) ayarlanması')

from keras.callbacks import EarlyStopping, LearningRateScheduler

earlystop = EarlyStopping(patience= 5)
    
lr_callback = LearningRateScheduler(lrfn, verbose = True)

callbacks = [earlystop, lr_callback]

In [ ]:
resNet_model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


resnet_history = resNet_model.fit_generator(res_train_data, epochs = 15,
                          validation_data = res_valid_data,
                          validation_steps= valid_images//batch_size,
                          steps_per_epoch= train_images//batch_size,
                          callbacks = callbacks)


In [ ]:
hist2 = resnet_history.history
fig = plt.figure(figsize = (15,5))

Epochs =  range(len(hist2['loss']))

fig.add_subplot(1, 2 ,1)
sns.lineplot(x = Epochs, y = hist2['val_loss'], label = 'Validation Loss')
sns.lineplot(x = Epochs, y = hist2['loss'], label = 'Training_loss')

fig.add_subplot(1,2,2)
sns.lineplot(x = Epochs, y = hist2['val_accuracy'], label = 'Validation accuracy')
sns.lineplot(x = Epochs, y = hist2['accuracy'], label = 'Training_accuracy')


In [ ]:
res_pred = resNet_model.predict_generator(res_test_data)
test_df['res_pred'] = np.argmax(res_pred, axis = -1)
labels = dict((v,k) for k,v in res_train_data.class_indices.items())

test_df['res_pred'] = test_df['res_pred'].map(labels)


In [ ]:
test_df.head(50)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

res_cf_matrix = confusion_matrix(test_df['category'],test_df['res_pred'])


labels = [ 'True Neg','False Pos','False Neg','True Pos']
categories = ['Cat', 'Dog']
make_confusion_matrix(res_cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      title = 'ResNet comfusion matrix')

res_matrix = classification_report(test_df['category'],test_df['res_pred'])
print('Classification report : \n',res_matrix)

# 9. Test Verisi ile Son Tahminler

In [ ]:
# generating an dataframe iterator for test dataset

vani_sub_aug_map = ImageDataGenerator()
res_sub_aug_map = ImageDataGenerator(preprocessing_function = preprocess_input)

vani_sub_data = vani_sub_aug_map.flow_from_dataframe(
             sub_df, test_dir,
             x_col = 'filename',
             y_col = None,
             class_mode = None,
             target_size = (img_size, img_size),
             shuffle = False)


res_sub_data = res_sub_aug_map.flow_from_dataframe(
             sub_df, test_dir,
             x_col = 'filename',
             y_col = None,
             class_mode = None,
             target_size = (img_size, img_size),
             shuffle = False)

In [ ]:
vani_pred_sub = vani_model.predict_generator(vani_sub_data)
sub_df['vani_pred_sub'] = np.argmax(vani_pred_sub, axis = -1)
labels = dict((v,k) for k,v in res_train_data.class_indices.items())
sub_df['vani_pred_sub'] = sub_df['vani_pred_sub'].map(labels)


res_pred_sub = resNet_model.predict_generator(res_sub_data)
sub_df['res_pred_sub'] = np.argmax(res_pred_sub, axis = -1)
labels = dict((v,k) for k,v in res_train_data.class_indices.items())
sub_df['res_pred_sub'] = sub_df['res_pred_sub'].map(labels)

sub_df.head()

In [ ]:
pred_sample = sub_df.sample(18)
pred_sample.reset_index(drop = True, inplace = True)
plt.figure(figsize=(12,24))
for index, row in pred_sample.iterrows():
    filename = row['filename']
    vani_pred = row['vani_pred_sub']
    res_pred = row['res_pred_sub']
    img = load_img( test_dir + filename, target_size= (img_size, img_size))
    plt.subplot(6,3, index+1)
    plt.imshow(img)
    plt.text(130, 175, 'vanila_pred: {}'.format(vani_pred), color='lightgreen',fontsize= 11, bbox=dict(facecolor='black', alpha=0.9))
    plt.text(130, 200, 'resNet_pred: {}'.format(res_pred), color='red',fontsize= 11, bbox=dict(facecolor='black', alpha=0.9))
    plt.title(filename.split('.')[0])
plt.tight_layout()
#plt.subplots_adjust( wspace=0, hspace= 1)
plt.show()
   

In [ ]:
sub_df

### Reading predictions to submission.csv file

In [ ]:
sub_df['res_pred_sub'].replace({'cat': 0, 'dog': 1},  inplace = True)
sub_df.rename(columns = {'res_pred_sub': 'label'}, inplace = True)
sub_df['filename'] = sub_df['filename'].str.split('.').str[0]
sub_df.rename(columns = {'filename': 'id'}, inplace = True)
sub_df.drop(['vani_pred_sub'], axis=1, inplace=True)
sub_df.to_csv('submission.csv', index=False)

In [ ]:
sub_df